# 3D GOL

In [1]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import subprocess

In [11]:
class GameOfLife3D:
    """
    Implements a 3D Cellular Automaton with 3 states (0: Dead, 1: Type A, 2: Type B) with periodic boundary conditions.
    The rules are:
    Currently, it outputs a textfile and a frame (3D Proj) image every step.
    """
    def __init__(self, size_l, size_w, size_h):
        if not (isinstance(size_l, int) and size_l > 0 and
                  isinstance(size_w, int) and size_w > 0 and
                  isinstance(size_h, int) and size_h > 0):
            raise ValueError("Grid dim must be positive integers.")

        self.size_l = size_l
        self.size_w = size_w
        self.size_h = size_h
        self.grid = np.zeros((size_l, size_w, size_h), dtype=np.int8)
        print(f"Initialized grid with shape: {self.grid.shape}")

    def randomize(self, density_a=0.1, density_b=0.1):
        """
        Start randomly with Type A and Type B cells based on densities.
        """
        if not (0 <= density_a <= 1 and 0 <= density_b <= 1):
             raise ValueError("Densities must be between 0 and 1.")
        if density_a + density_b > 1:
            raise ValueError("Sum of densities (density_a + density_b) cannot exceed 1.")
            # Can be less
        total_cells = self.size_l * self.size_w * self.size_h
        num_a = int(total_cells * density_a)
        num_b = int(total_cells * density_b)
        num_dead = total_cells - num_a - num_b

        states = [1] * num_a + [2] * num_b + [0] * num_dead
        random.shuffle(states)

        self.grid = np.array(states, dtype=np.int8).reshape(self.grid.shape)
        print(f"Randomized grid: ~{density_a*100:.1f}% Type A, ~{density_b*100:.1f}% Type B.")

    def _count_neighbors(self, x, y, z):
        """Counts Type A and Type B neighbors using periodic boundaries."""
        # TODO: should we use periodic boundaries?
        count_a = 0
        count_b = 0
        for i_offset in [-1, 0, 1]:
            for j_offset in [-1, 0, 1]:
                for k_offset in [-1, 0, 1]:
                    if i_offset == 0 and j_offset == 0 and k_offset == 0:
                        continue
                    nx = (x + i_offset) % self.size_l
                    ny = (y + j_offset) % self.size_w
                    nz = (z + k_offset) % self.size_h
                    neighbor_state = self.grid[nx, ny, nz]
                    if neighbor_state == 1:
                        count_a += 1
                    elif neighbor_state == 2:
                        count_b += 1
        return count_a, count_b

    def _apply_rules(self, current_state, count_a, count_b):
        """The rule currently is: """
        # 3 neighbors of A and 0 neighbors of B will create type A
        # 3 neighbors of B and 0 neighbors of A will create type B
        # Type A cell lives with 2 or 3 living neighbors
        # Type B cell lives with 3 or 4 living neighbors
        # TODO: should be given more thoughts

        next_state = current_state
        live_neighbors = count_a + count_b

        if current_state == 0: # Dead cell
            if count_a == 3 and count_b == 0: next_state = 1 # Birth of Type A
            elif count_b == 3 and count_a == 0: next_state = 2 # Birth of Type B
        elif current_state == 1: # Type A cell
            # Type A cell survices with 2 or 3 live neighbors
            if not (live_neighbors == 2 or live_neighbors == 3): next_state = 0 # Dies
        elif current_state == 2: # Type B cell
             # Type B survices with 3 or 4 live neighbors
            if not (live_neighbors == 3 or live_neighbors == 4): next_state = 0 # Dies
        return next_state

    def step(self):
        """Advances the game state by one time step."""
        new_grid = np.zeros_like(self.grid, dtype=np.int8)
        for x in range(self.size_l):
            for y in range(self.size_w):
                for z in range(self.size_h):
                    current_state = self.grid[x, y, z]
                    count_a, count_b = self._count_neighbors(x, y, z)
                    new_grid[x, y, z] = self._apply_rules(current_state, count_a, count_b)
        self.grid = new_grid

    def get_grid(self):
        """Returns a copy of the current grid state."""
        return self.grid.copy()

    def get_state_counts(self):
        """Counts the total number of cells in each state."""
        unique, counts = np.unique(self.grid, return_counts=True)
        # Ensure all states (0, 1, 2) are present in the output dict
        state_counts = {0: 0, 1: 0, 2: 0}
        state_counts.update(dict(zip(unique, counts)))
        return state_counts

    # visualization
    def visualize_and_save_plot(self, filename, step_number, view_elev=20., view_azim=-65):
        """
        Generates a 3D scatter plot of the current grid state using Matplotlib
        and saves it to a file. Only live cells are plotted.

        Args:
            filename (str): The path to save the image file (e.g., 'frames/frame_001.png').
            step_number (int): The current step number (for the plot title).
            view_elev (float): Elevation angle for the 3D plot view.
            view_azim (float): Azimuth angle for the 3D plot view.
        """
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Get coordinates and states of live cells
        live_cells = np.argwhere(self.grid > 0) # Indices where state is 1 or 2
        if live_cells.size == 0:
             print(f"Step {step_number}: No live cells to plot.")
             # Still save an empty plot for consistent frame count
        else:
            x_coords, y_coords, z_coords = live_cells[:, 0], live_cells[:, 1], live_cells[:, 2]
            states = self.grid[x_coords, y_coords, z_coords]
            # Blue for Type A (1), Green for Type B (2)
            colors = ['#3b82f6' if s == 1 else '#10b981' for s in states]
            # Create the scatter plot using squares ('s') as markers
            ax.scatter(x_coords, y_coords, z_coords, c=colors, marker='s', s=60, depthshade=True)
        ax.set_xlim(0, self.size_l)
        ax.set_ylim(0, self.size_w)
        ax.set_zlim(0, self.size_h)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis.pane.set_edgecolor('w')
        ax.yaxis.pane.set_edgecolor('w')
        ax.zaxis.pane.set_edgecolor('w')
        ax.grid(True, which='both', axis='both', linestyle='--', linewidth=0.5, color='grey')

        ax.set_title(f"3D Game of Life - Step {step_number}")
        ax.view_init(elev=view_elev, azim=view_azim)

        try:
            plt.savefig(filename, dpi=120, bbox_inches='tight', facecolor='white') # Adjust dpi, add background
            # print(f"Saved plot: {filename}") # Reduce console noise
        except Exception as e:
            print(f"Error saving plot {filename}: {e}")
        finally:
            plt.close(fig) # Close the figure to free memory

    def save_state_to_text(self, filename):
        """
        Saves the entire current grid state to a text file, slice by slice.

        Args:
            filename (str): The path to save the text file (e.g., 'states/state_001.txt').
        """
        try:
            with open(filename, 'w') as f:
                 f.write(f"# Grid State: {self.size_l}x{self.size_w}x{self.size_h}\n")
                 # Save each z-slice separately for better readability
                 for z in range(self.size_h):
                     f.write(f"# Slice z={z}\n")
                     np.savetxt(f, self.grid[:, :, z], fmt='%d') # Save slice data
                     f.write("\n") # Add newline between slices
            # print(f"Saved state: {filename}") # Reduce console noise
        except Exception as e:
            print(f"Error saving state {filename}: {e}")

In [12]:
# --- Helper Function for Running Simulation and Saving ---
def run_simulation_and_save(game_instance, num_steps, frames_dir, states_dir, base_filename_tag="gol"):
    """
    saves visualization frames of GOL and state files for each step.
    """
    print(f"\nStarting simulation run '{base_filename_tag}' for {num_steps} steps...")
    print(f"Output frames to: {frames_dir}")
    print(f"Output states to: {states_dir}")

    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(states_dir, exist_ok=True)
    for step_num in range(num_steps + 1):
        frame_filename = os.path.join(frames_dir, f"{base_filename_tag}_frame_{str(step_num).zfill(4)}.png")
        state_filename = os.path.join(states_dir, f"{base_filename_tag}_state_{str(step_num).zfill(4)}.txt")

        game_instance.visualize_and_save_plot(frame_filename, step_num)
        game_instance.save_state_to_text(state_filename)

        # Print progress and state counts periodically or at the end
        counts = game_instance.get_state_counts()
        print(f"\rStep {step_num}/{num_steps} | Counts: A={counts[1]}, B={counts[2]}, Dead={counts[0]}", end="")

        # Advance to the next step (unless it's the last iteration)
        if step_num < num_steps:
            game_instance.step()

    print(f"\nSimulation run '{base_filename_tag}' finished.")

    # Construct the suggested command using the actual output directory and filename pattern
    ffmpeg_input_pattern = os.path.join(frames_dir, f"{base_filename_tag}_frame_%04d.png")
    output_video_filename = f"{base_filename_tag}_video.mp4"
    print(f"   ffmpeg -framerate 10 -i \"{ffmpeg_input_pattern}\" -c:v libx264 -pix_fmt yuv420p \"{output_video_filename}\"")
    print("--------------------------------------------------")

In [13]:
if __name__ == "__main__":
    GRID_L, GRID_W, GRID_H = 20, 20, 20 # Grid dimensions
    NUM_STEPS = 10
    INITIAL_DENSITY_A = 0.04           # Initial density for Type A
    INITIAL_DENSITY_B = 0.02           # Initial density for Type B

    SIMULATION_TAG = "my_3d_gol_run_1" # Specific tag for this run's files
    OUTPUT_FRAMES_DIR = os.path.join("output", SIMULATION_TAG, "frames") # Subfolder for frames
    OUTPUT_STATES_DIR = os.path.join("output", SIMULATION_TAG, "states") # Subfolder for states

    # --- Initialize Game ---
    game = GameOfLife3D(size_l=GRID_L, size_w=GRID_W, size_h=GRID_H)
    game.randomize(density_a=INITIAL_DENSITY_A, density_b=INITIAL_DENSITY_B)

    # --- Run Simulation and Save Outputs using the helper function ---
    run_simulation_and_save(
        game_instance=game,
        num_steps=NUM_STEPS,
        frames_dir=OUTPUT_FRAMES_DIR,
        states_dir=OUTPUT_STATES_DIR,
        base_filename_tag=SIMULATION_TAG
    )

    print("\nScript finished.")

Initialized grid with shape: (20, 20, 20)
Randomized grid: ~4.0% Type A, ~2.0% Type B.

Starting simulation run 'my_3d_gol_run_1' for 10 steps...
Output frames to: output/my_3d_gol_run_1/frames
Output states to: output/my_3d_gol_run_1/states
Step 10/10 | Counts: A=1210, B=43, Dead=6747
Simulation run 'my_3d_gol_run_1' finished.
   ffmpeg -framerate 10 -i "output/my_3d_gol_run_1/frames/my_3d_gol_run_1_frame_%04d.png" -c:v libx264 -pix_fmt yuv420p "my_3d_gol_run_1_video.mp4"
--------------------------------------------------

Script finished.


In [14]:
!ffmpeg -framerate 10 -i output/my_3d_gol_run_1/frames/my_3d_gol_run_1_frame_%04d.png -c:v libx264 -pix_fmt yuv420p my_3d_gol_run_1_video.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# 2D GOL

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import subprocess

def num_live_neigh(mat, i, j):
    """Counts live neighbors for a cell (i, j)."""
    x_size, y_size = mat.shape
    count = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0: # doesn't count the cell itself
                continue
            # still use periodic condition
            ni = (i + dx) % x_size
            nj = (j + dy) % y_size
            # if not periodic
            # if 0 <= ni < x_size and 0 <= nj < y_size:
            count += mat[ni, nj]
    return count

def step_2d(mat):
    """Advances by one step."""
    x_size, y_size = mat.shape
    next_gen = np.copy(mat)
    for i in range(x_size):
        for j in range(y_size):
            live_neighbors = num_live_neigh(mat, i, j)
            if mat[i, j] == 1: # cell state at time t (alive)
                if live_neighbors < 2 or live_neighbors > 3:
                    next_gen[i, j] = 0 # Dies
                # else: lives (already 1)
            else: # cell is dead at time t
                if live_neighbors == 3:
                    next_gen[i, j] = 1 # Becomes alive
    return next_gen # Return the next state grid


In [ ]:
def save_frame_2d(mat, filename, step_number):
    """
    Saves the current 2D grid state as a grayscale image using Matplotlib.

    Args:
        mat (np.ndarray): The 2D grid array.
        filename (str): The path to save the image file (e.g., 'frames_2d/frame_001.png').
        step_number (int): The current step number (for the plot title).
    """
    fig = plt.figure(figsize=(8, 8)) # Adjust figure size as needed
    ax = fig.add_subplot(111)
    # Use imshow for 2D grids, cmap='gray' for black/white, vmin/vmax ensure consistency
    ax.imshow(mat, cmap='gray', vmin=0, vmax=1, interpolation='nearest')
    ax.set_title(f"2D Game of Life - Step {step_number}")
    ax.set_xticks([]) # Hide axes ticks for cleaner look
    ax.set_yticks([])

    try:
        plt.savefig(filename, dpi=100, bbox_inches='tight')
        # print(f"Saved plot: {filename}") # Reduce console noise
    except Exception as e:
        print(f"Error saving plot {filename}: {e}")
    finally:
        plt.close(fig) # Close the figure to free memory

def save_state_2d(mat, filename):
    """
    Saves the entire current 2D grid state to a text file.

    Args:
        mat (np.ndarray): The 2D grid array.
        filename (str): The path to save the text file (e.g., 'states_2d/state_001.txt').
    """
    try:
        with open(filename, 'w') as f:
            f.write(f"# Grid State 2D: {mat.shape[0]}x{mat.shape[1]}\n")
            np.savetxt(f, mat, fmt='%d') # Save the 2D array directly
        # print(f"Saved state: {filename}") # Reduce console noise
    except Exception as e:
        print(f"Error saving state {filename}: {e}")

In [ ]:
def run_gol_2d_and_save(initial_mat, num_steps, frames_dir, states_dir, base_filename_tag="gol_2d"):
    """
    Runs the 2D GoL simulation, saves visualization frames and state files for each step.

    Args:
        initial_mat (np.ndarray): The starting 2D grid state.
        num_steps (int): The total number of steps to simulate.
        frames_dir (str): Directory path to save image frames.
        states_dir (str): Directory path to save text state files.
        base_filename_tag (str): A prefix for the output filenames.
    """
    print(f"\nStarting 2D simulation run '{base_filename_tag}' for {num_steps} steps...")
    print(f"Output frames to: {frames_dir}")
    print(f"Output states to: {states_dir}")

    # Create output directories if they don't exist
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(states_dir, exist_ok=True)

    current_mat = np.copy(initial_mat)
    prev_mat = None
    prev_prev_mat = None

    for step_num in range(num_steps + 1):
        frame_filename = os.path.join(frames_dir, f"{base_filename_tag}_frame_{str(step_num).zfill(4)}.png")
        state_filename = os.path.join(states_dir, f"{base_filename_tag}_state_{str(step_num).zfill(4)}.txt")
        save_frame_2d(current_mat, frame_filename, step_num)
        save_state_2d(current_mat, state_filename)

        live_count = np.sum(current_mat)
        total_cells = current_mat.size
        print(f"\rStep {step_num}/{num_steps} | Live Cells: {live_count}/{total_cells}      ", end="")

        # Stability check
        if prev_mat is not None and prev_prev_mat is not None:
             # Check for static pattern (current == previous) or oscillator (current == prev_prev)
            if np.array_equal(current_mat, prev_mat) or np.array_equal(current_mat, prev_prev_mat):
                print(f"\nPattern stabilized or entered cycle at step {step_num}. Stopping.")
                break # Stop simulation

        if step_num < num_steps:
            next_mat = step_2d(current_mat)
            prev_prev_mat = prev_mat
            prev_mat = current_mat
            current_mat = next_mat
        else:
            pass


    print(f"\nSimulation run '{base_filename_tag}' finished.")

    ffmpeg_input_pattern = os.path.normpath(os.path.join(frames_dir, f"{base_filename_tag}_frame_%04d.png"))
    output_video_filename = os.path.normpath(f"{base_filename_tag}_video.mp4")


In [21]:
if __name__ == "__main__":
    GRID_SIZE_X, GRID_SIZE_Y = 100, 100
    NUM_STEPS = 200
    INITIAL_DENSITY = 0.2

    SIMULATION_TAG = "my_2d_gol_run"
    OUTPUT_BASE_DIR = "output_2d"
    OUTPUT_FRAMES_DIR = os.path.join(OUTPUT_BASE_DIR, SIMULATION_TAG, "frames")
    OUTPUT_STATES_DIR = os.path.join(OUTPUT_BASE_DIR, SIMULATION_TAG, "states")

    initial_grid = np.random.choice([0, 1], size=(GRID_SIZE_X, GRID_SIZE_Y), p=[1 - INITIAL_DENSITY, INITIAL_DENSITY])
    print(f"Initialized 2D grid with shape: {initial_grid.shape}")
    print(f"Initial live cell density: ~{np.mean(initial_grid)*100:.1f}%")

    run_gol_2d_and_save(
        initial_mat=initial_grid,
        num_steps=NUM_STEPS,
        frames_dir=OUTPUT_FRAMES_DIR,
        states_dir=OUTPUT_STATES_DIR,
        base_filename_tag=SIMULATION_TAG
    )

    print("\nScript finished.")

Initialized 2D grid with shape: (100, 100)
Initial live cell density: ~20.1%

Starting 2D simulation run 'my_2d_gol_run' for 200 steps...
Output frames to: output_2d/my_2d_gol_run/frames
Output states to: output_2d/my_2d_gol_run/states
Step 200/200 | Live Cells: 743/10000      
Simulation run 'my_2d_gol_run' finished.

Script finished.


In [23]:
!ffmpeg -framerate 10 -i "output_2d/my_2d_gol_run/frames/my_2d_gol_run_frame_%04d.png" -c:v libx264 -pix_fmt yuv420p "my_2d_gol_run_1_video.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [25]:
!git status

fatal: not a git repository (or any of the parent directories): .git
